In [1]:
import warnings
warnings.filterwarnings('ignore')
    
import arviz as az
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
from gala.units import UnitSystem
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pymc3 as pm
import pymc3_ext as pmx
from tqdm.notebook import tqdm
import astropy.table as at
from aesara_theano_fallback import tensor as tt
import exoplanet.units as xu
import json

from yellowcard.model_pymc3 import TimingArgumentModelPymc3

In [2]:
plt.rcParams.update({"text.usetex":False})

### To get mass as a function of travel velocity

In [35]:
# # ------------------------------------------- #
# # this bit of the code runs pymc3 for various #
# # values of the travel velocity of the disk   #
# # ------------------------------------------- #
# # includes ~0.1% error on velocity measure    #
# # ------------------------------------------- #

# # modelChoice = "vdm2012"
# modelChoice = "fiducial2021"

# vtravel = 70
# vtravel_err = 1e-3

# if vtravel == 0:
#     model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{modelChoice}.ecsv", 
#                                               include_vtravel=False)
# else:
#     model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{modelChoice}.ecsv", 
#                                               include_vtravel=True,
#                                               vtravel_mag=vtravel,
#                                               vtravel_mag_err=vtravel_err)
# with model.pymc3_model:
#     start = {
#     'r': 770.,  
#     'M': 4.2, 
#     'eta': 4.5,
#     'ln(1-e)': -3,
#     'alpha': 0.5
#     }
#     res = pmx.optimize(start=start)
#     print(repr(res))

#     samples = pmx.sample(return_inferencedata=True, 
#                          start=res, tune=4000, draws=2000)

# samples.to_json(f"../pymc3-data/{modelChoice}_vtravel_{vtravel}.json")
# print(f"exported to ../pymc3-data/{modelChoice}_vtravel_{vtravel}.json")
# print("ALL. DAMN. DONE.")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -3093.899065450101 -> 45.09710149881158


{'r_interval__': array(-2.63148973), 'M_interval__': array(-1.40578006), 'eta_angle__': array([-3.73934671, -2.45301572]), 'ln(1-e)_interval__': array(1.92396008), 'alpha_angle__': array([ 4.36693298, -0.96431133]), 'vtravel_mag': array(71.58985155), 'vtravel_lon': array(0.96429555), 'vtravel_lat': array(-0.59958543), 'r': array(764.67695625), 'M': array(4.33955984), 'eta': array(-2.15137872), 'sineta': array(-0.83614334), 'coseta': array(-0.54851099), 'ln(1-e)': array(-1.27420616), 'e': array(0.72034712), 'a': array(548.10903786), 'vscale': array(188.72234554), 'vrad': array(-81.4769659), 'vtan': array(93.82746352), 'tperi': array(13.74937331), 'sun_m31_dist': array(760.7340453), 'm31_icrs_xyz': array([561.8725182 , 106.01161022, 501.77395255]), 'alpha': array(1.78812977), 'model_pmra': array(0.23536552), 'model_pmdec': array(-0.18476058), 'model_rv': array(-308.86732915)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 76 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There were 25 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


exported to ../pymc3-data/fiducial2021_vtravel_100.json
ALL. DAMN. DONE.


----
# Mass vs. vtravel

In [5]:
# ------------------------------------------- #
# this bit of the code runs pymc3 for various #
# values of the travel velocity of the disk   #
# ------------------------------------------- #
# includes ~1% error on velocity measure      #
# ------------------------------------------- #

modelChoices = ["fiducial2021","vdm2012","Dataset3"]

# travel velocities of MW disk wrt halo:
vtravels = np.concatenate( (np.arange(0,105,5), np.array([32])) )
vtravel_err = 1e-2

for dataset in modelChoices:
    
#     if dataset=="vdm2012":
#         continue
    for vel in vtravels:
        print(f'working on {dataset} travel velocity {vel}')
        if vel == 0:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=False)
        elif vel == 32:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True)
            
        else:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=vel,
                                                      vtravel_mag_err=vtravel_err)
        with model.pymc3_model:
            start = {
            'r': 770.,  
            'M': 4.2, 
            'eta': 4.5,
            'ln(1-e)': -3,
            'alpha': 0.5
            }
            res = pmx.optimize(start=start)
            print(repr(res))

            samples = pmx.sample(return_inferencedata=True, 
                                 start=res, chains=4, tune=4000, draws=8000)

        samples.to_json(f"../pymc3-data/{dataset}_vtravel_{vel}.json")
print("ALL. DAMN. DONE.")

working on fiducial2021


optimizing logp for variables: [alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -36711.301122280645 -> 38.989192535760075


{'r_interval__': array(-2.63042895), 'M_interval__': array(-1.11771762), 'eta_angle__': array([-4.05414096, -1.88784031]), 'ln(1-e)_interval__': array(1.44220095), 'alpha_angle__': array([3.69991139, 2.51210185]), 'r': array(765.33499755), 'M': array(5.30548028), 'eta': array(-2.0065943), 'sineta': array(-0.90653348), 'coseta': array(-0.42213393), 'ln(1-e)': array(-1.9120475), 'e': array(0.8522225), 'a': array(562.84894683), 'vscale': array(205.92088205), 'vrad': array(-116.99775173), 'vtan': array(79.23032065), 'tperi': array(13.80099933), 'sun_m31_dist': array(761.39211509), 'm31_icrs_xyz': array([562.35856366, 106.10331512, 502.20801   ]), 'alpha': array(0.97432932), 'model_pmra': array(0.24910958), 'model_pmdec': array(-0.18807238), 'model_rv': array(-307.83390538)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, ln(1-e), eta, M, r]


...

message: Optimization terminated successfully.
logp: -37118.903617904136 -> 38.34312965593742


{'r_interval__': array(-2.63115217), 'M_interval__': array(-1.43946087), 'eta_angle__': array([-3.86554089, -2.24890943]), 'ln(1-e)_interval__': array(1.27867693), 'alpha_angle__': array([ 0.79727308, -4.40049493]), 'vtravel_mag': array(32.9760154), 'vtravel_lon': array(0.9697035), 'vtravel_lat': array(-0.58695512), 'r': array(764.88629105), 'M': array(4.23676256), 'eta': array(-2.09771394), 'sineta': array(-0.86436122), 'coseta': array(-0.50287144), 'ln(1-e)': array(-2.17775523), 'e': array(0.88670443), 'a': array(529.00420046), 'vscale': array(189.81103996), 'vrad': array(-100.61387257), 'vtan': array(60.69348933), 'tperi': array(13.80097898), 'sun_m31_dist': array(760.94338917), 'm31_icrs_xyz': array([562.02713803, 106.0407832 , 501.91203406]), 'alpha': array(2.96235887), 'model_pmra': array(0.14264464), 'model_pmdec': array(-0.10241777), 'model_rv': array(-307.83897173)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '28397' (I am process '28466')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 8_000 draw iterations (16_000 + 32_000 draws total) took 214 seconds.
There were 61 divergences after tuning. Increase `target_accept` or reparameterize.
There were 187 divergences after tuning. Increase `target_accept` or reparameterize.
There were 263 divergences after tuning. Increase `target_accept` or reparameterize.
There were 101 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


ALL. DAMN. DONE.


---
# Mass error vs. pm error

In [6]:
# ------------------------------------------- #
# this bit of the code runs pymc3 for various #
# values of the travel velocity of the disk   #
# ------------------------------------------- #
# includes ~0.1% error on velocity measure    #
# ------------------------------------------- #

modelChoices = ["fiducial2021","vdm2012","Dataset3"]
pm_errs = np.array([1,2,4,8,16])

for dataset in modelChoices:
#     if dataset == "fiducial2021":
#         continue
    for improv in pm_errs:
        print(f'working on {dataset} and error {improv}')
        model = (TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True))
        
#         params = model_default.__dict__
#         print(params["pm_err"])
#         params["pm_err"] /= improv
#         print(params["pm_err"])
        
#         model = TimingArgumentModelPymc3(**params)
        
        with model.pymc3_model:
            pm.set_data({
            'obs_pmra_err':model.pm_err[0].to_value(model.units['angular speed'])/improv,
            'obs_pmdec_err':model.pm_err[1].to_value(model.units['angular speed'])/improv
            })
            start = {
            'r': 770.,  
            'M': 4.2, 
            'eta': 4.5,
            'ln(1-e)': -3,
            'alpha': 0.5
            }
            res = pmx.optimize(start=start)
            print(repr(res))

            samples = pmx.sample(return_inferencedata=True, 
                                 start=res, chains=4, tune=4000, draws=8000)

        samples.to_json(f"../pymc3-data/{dataset}_pmimprov_{improv}.json")
        print(improv,np.std(samples.posterior.M))
print("ALL. DAMN. DONE.")

working on fiducial2021 and error 1


optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -37117.429949243065 -> 38.49760686496404


{'r_interval__': array(-2.63035545), 'M_interval__': array(-1.30340124), 'eta_angle__': array([-3.91249257, -2.16619524]), 'ln(1-e)_interval__': array(1.59111095), 'alpha_angle__': array([4.40642215, 0.76383503]), 'vtravel_mag': array(32.74757696), 'vtravel_lon': array(0.97417956), 'vtravel_lat': array(-0.60136264), 'r': array(765.38061284), 'M': array(4.66506644), 'eta': array(-2.07644608), 'sineta': array(-0.87485994), 'coseta': array(-0.48437598), 'ln(1-e)': array(-1.69227651), 'e': array(0.81590006), 'a': array(548.58034721), 'vscale': array(195.58824984), 'vrad': array(-100.06473307), 'vtan': array(81.05472651), 'tperi': array(13.80098331), 'sun_m31_dist': array(761.43773236), 'm31_icrs_xyz': array([562.39225624, 106.10967208, 502.23809878]), 'alpha': array(1.3991561), 'model_pmra': array(0.24126095), 'model_pmdec': array(-0.18970668), 'model_rv': array(-307.83916646)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


...

message: Optimization terminated successfully.
logp: -37990.93237685918 -> 43.58379990822316


{'r_interval__': array(-2.63159459), 'M_interval__': array(-1.54126985), 'eta_angle__': array([-3.82906633, -2.31046551]), 'ln(1-e)_interval__': array(0.94340234), 'alpha_angle__': array([ 0.83649245, -4.39320844]), 'vtravel_mag': array(33.18981151), 'vtravel_lon': array(0.96512059), 'vtravel_lat': array(-0.58249496), 'r': array(764.61194209), 'M': array(3.93883966), 'eta': array(-2.1137134), 'sineta': array(-0.85620526), 'coseta': array(-0.5166358), 'ln(1-e)': array(-2.80213599), 'e': array(0.93931969), 'a': array(514.79099084), 'vscale': array(185.52508894), 'vrad': array(-100.45783826), 'vtan': array(42.84904169), 'tperi': array(13.80097691), 'sun_m31_dist': array(760.66902832), 'm31_icrs_xyz': array([561.82449714, 106.00254982, 501.73106789]), 'alpha': array(2.95343906), 'model_pmra': array(0.16612795), 'model_pmdec': array(-0.0980525), 'model_rv': array(-307.83850651)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '39439' (I am process '39470')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 8_000 draw iterations (16_000 + 32_000 draws total) took 111 seconds.
There were 28 divergences after tuning. Increase `target_accept` or reparameterize.
There were 149 divergences after tuning. Increase `target_accept` or reparameterize.
There were 9 divergences after tuning. Increase `target_accept` or reparameterize.
There were 38 divergences after tuning. Increase `target_accept` or reparameterize.


16 <xarray.DataArray 'M' ()>
array(0.20297175)
ALL. DAMN. DONE.


In [24]:
model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=32,
                                                      vtravel_mag_err=1e-3)
with model.pymc3_model:
    start = {
    'r': 770.,  
    'M': 4.2, 
    'eta': 4.5,
    'ln(1-e)': -3,
    'alpha': 0.5
    }
    res = pmx.optimize(start=start)
    print(repr(res))

    samples = pmx.sample(return_inferencedata=True, 
                         start=res, tune=4000, draws=2000);
    
for name in ['M', 'r']:
    chain = np.asarray(getattr(samples.posterior, name)).ravel()
    print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -2260.4941670852018 -> 45.072829861027785


{'r_interval__': array(-2.63133102), 'M_interval__': array(-1.29030243), 'eta_angle__': array([-3.91867302, -2.1549946 ]), 'ln(1-e)_interval__': array(1.58026439), 'alpha_angle__': array([4.40737222, 0.75833377]), 'vtravel_mag': array(32.72678928), 'vtravel_lon': array(0.97436866), 'vtravel_lat': array(-0.60140203), 'r': array(764.77537367), 'M': array(4.70813169), 'eta': array(-2.07358555), 'sineta': array(-0.87624193), 'coseta': array(-0.48187144), 'ln(1-e)': array(-1.70758042), 'e': array(0.81869606), 'a': array(548.42018359), 'vscale': array(196.51764549), 'vrad': array(-101.09456679), 'vtan': array(80.9216582), 'tperi': array(13.74967037), 'sun_m31_dist': array(760.83246699), 'm31_icrs_xyz': array([561.94521173, 106.02532571, 501.8388707 ]), 'alpha': array(1.40040443), 'model_pmra': array(0.24125054), 'model_pmdec': array(-0.18972886), 'model_rv': array(-308.86151893)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '18098' (I am process '18123')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 84 seconds.
There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


M: 4.70 +/- 0.67
r: 764.58 +/- 10.89


In [25]:
model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=32,
                                                      vtravel_mag_err=4)
with model.pymc3_model:
    start = {
    'r': 770.,  
    'M': 4.2, 
    'eta': 4.5,
    'ln(1-e)': -3,
    'alpha': 0.5
    }
    res = pmx.optimize(start=start)
    print(repr(res))

    samples = pmx.sample(return_inferencedata=True, 
                         start=res, tune=4000, draws=2000);
    
for name in ['M', 'r']:
    chain = np.asarray(getattr(samples.posterior, name)).ravel()
    print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -2268.7882167253038 -> 36.77879750386076


{'r_interval__': array(-2.63133109), 'M_interval__': array(-1.29041216), 'eta_angle__': array([-3.91856573, -2.15518963]), 'ln(1-e)_interval__': array(1.58042578), 'alpha_angle__': array([4.40759734, 0.75702452]), 'vtravel_mag': array(32.75084106), 'vtravel_lon': array(0.97436383), 'vtravel_lat': array(-0.60140522), 'r': array(764.77532903), 'M': array(4.70776959), 'eta': array(-2.07363532), 'sineta': array(-0.87621794), 'coseta': array(-0.48191506), 'ln(1-e)': array(-1.70735189), 'e': array(0.81865462), 'a': array(548.41396286), 'vscale': array(196.51120286), 'vrad': array(-101.0822282), 'vtan': array(80.92641658), 'tperi': array(13.74967015), 'sun_m31_dist': array(760.83242234), 'm31_icrs_xyz': array([561.94517875, 106.02531949, 501.83884125]), 'alpha': array(1.40070148), 'model_pmra': array(0.24124424), 'model_pmdec': array(-0.18972762), 'model_rv': array(-308.8615226)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '18261' (I am process '18286')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 81 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There were 34 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


M: 4.66 +/- 0.64
r: 764.47 +/- 10.78


___

___
# Graveyard ⚰️☠️

In [6]:
# truths = simulated_truths.copy()
# for k in list(samples.posterior.data_vars):
#     truths.setdefault(k, np.nan)
# truths = None    

In [7]:
# for name in ['M', 'r']:
#     chain = np.asarray(getattr(samples.posterior, name)).ravel()
#     print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

In [8]:
# pcl = np.percentile(np.ravel(samples.posterior.M), [16, 50, 84])
# err1, err2 = np.diff(pcl)
# print(f"{pcl[1]:.3f} +{err1:.3f} -{err2:.3f}")

In [9]:
# for name in ['M', 'r','e']:
#     chain = np.asarray(getattr(samples.posterior, name)).ravel()
#     print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")